In [1]:
! pip install selenium

In [2]:
import selenium
from selenium import webdriver as wb
import pandas as pd
import numpy as np
import datetime
import time
from datetime import date
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [3]:
%cd "C:\webdrivers"
driver = wb.Chrome("C:\\webdrivers\\chromedriver.exe")

C:\webdrivers


<ipython-input-3-6b16d037cd57>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = wb.Chrome("C:\\webdrivers\\chromedriver.exe")


In [4]:
month = [1,1,1,1,1,1,1]
month = [str(x).zfill(2) for x in month]
day = [4,5,6,7,8,9,10,11]
day = [str(x).zfill(2) for x in day]
year = [2022,2022,2022,2022,2022,2022,2022]
year = [str(x).zfill(4) for x in year]

In [5]:
# Creating an empty dataframe called 'Flight_Prices'
Flight_Prices = pd.DataFrame()
# Creating a for loop which will iterate through all the required pages of the website
for a,b,c in zip(day,month,year):
    driver.get("https://www.makemytrip.com/flight/search?tripType=O&itinerary=DEL-BOM-{}/{}/{}&paxType=A-1_C-0_I-0&cabinClass=E&sTime=1597828876664&forwardFlowRequired=true".format(a,b,c))
    time.sleep(15)
    
    lenOfPage = driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
    match=False
    while(match==False):
        lastCount = lenOfPage
        time.sleep(1)
        lenOfPage = driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
        if lastCount==lenOfPage:
            match=True
    
    time.sleep(60)
    # Extracting all the Airline names using xpath
    FlightName_elements = driver.find_elements_by_xpath("//div[@class='pull-left airways-info-sect']")
    FlightName_elements = [x.text for x in FlightName_elements]
    FlightName = [x.split('\n')[0] for x in FlightName_elements]
    FlightName = pd.Series(FlightName)
    
    # Extracting all the prices using xpath
    Price_elements = driver.find_elements_by_xpath("//span[@class='actual-price']")
    Price = [x.text for x in Price_elements]
    Price = [i for i in Price if i]
    Price = pd.Series(Price)
    
    # Extracting all the Source City details using xpath
    Fromcity_elements = driver.find_elements_by_xpath("//p[@class='dept-city']")
    Fromcity = [x.text for x in Fromcity_elements]
    Fromcity = pd.Series(Fromcity)
    
    # Extracting all the Destination City details using xpath
    Tocity_elements = driver.find_elements_by_xpath("//p[@class='arrival-city']")
    Tocity = [x.text for x in Tocity_elements]
    Tocity = pd.Series(Tocity)
    
    # Extracting all the Duration details using xpath
    Duration_elements = driver.find_elements_by_xpath("//p[@class='fli-duration']")
    Duration = [x.text for x in Duration_elements]
    Duration = pd.Series(Duration)
    
     # Extracting all the Departure time details using xpath
    Deptime_elements = driver.find_elements_by_xpath("//div[@class='dept-time']")
    Deptime = [x.text for x in Deptime_elements]
    Deptime = pd.Series(Deptime)
    
     # Extracting all the Arrival Time details using xpath
    Arrtime_elements = driver.find_elements_by_xpath("//p[@class='reaching-time append_bottom3']")
    Arrtime = [x.text for x in Arrtime_elements]
    Arrtime = [x.split("+", 1)[0] for x in Arrtime]
    Arrtime = pd.Series(Arrtime)
    
    Date_elements = driver.find_elements_by_xpath("//div[@class='item blue_active']")
    Date_elements = [x.text for x in Date_elements]
    x = [x.split(',', 1)[1] for x in Date_elements]
    Date = [i.split('\n', 1)[0] for i in x]
    Date = pd.Series(Date)
            
    # Combining all the lists into a dataframe called 'df'
    df = pd.DataFrame({'Date':Date,"Airline":FlightName,"From City":Fromcity, "To City":Tocity, "Departure Time":Deptime,"Arrival Time":Arrtime,"Flight Duration":Duration,"Price":Price})
    
    # We will append the results obtained from every page into the empty datafram created earlier called 'Flight_Prices'
    Flight_Prices = Flight_Prices.append(df) 
        
Flight_Prices[Flight_Prices.Date==""] = np.NaN
Flight_Prices.Date = Flight_Prices.Date.fillna(method='ffill')
Flight_Prices.Price = Flight_Prices.Price.astype(str).str.replace(",","").str.extract('(\d+)')
Flight_Prices

<ipython-input-5-86a0c10e6849>:19: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  FlightName_elements = driver.find_elements_by_xpath("//div[@class='pull-left airways-info-sect']")
<ipython-input-5-86a0c10e6849>:22: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  FlightName = pd.Series(FlightName)
<ipython-input-5-86a0c10e6849>:25: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  Price_elements = driver.find_elements_by_xpath("//span[@class='actual-price']")
<ipython-input-5-86a0c10e6849>:28: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  Price = pd.Series(Price)
<ipython-input-5-86a0c10e6849>:31: DeprecationWarning: find_elements_by_* commands 

,Date,Airline,From City,To City,Departure Time,Arrival Time,Flight Duration,Price


In [6]:
Flight_Prices['Price'] = pd.to_numeric(Flight_Prices['Price'])

In [7]:
Flight_Prices.loc[Flight_Prices.groupby('Date')['Price'].idxmin()]
Flight_Prices.drop_duplicates('Date')

,Date,Airline,From City,To City,Departure Time,Arrival Time,Flight Duration,Price
